In [1]:
import json
import pandas as pd

from tqdm.notebook import tqdm

In [2]:
df = pd.read_json("./input.json")

In [3]:
title_column = df['title']
title_column = title_column.unique()
title_column

array(['一些', '一击', '一团气', ..., '鱼骨', '麻烦', '鼓掌欢迎'], dtype=object)

In [4]:
%pip install --upgrade openai

Note: you may need to restart the kernel to use updated packages.


In [5]:
from openai import OpenAI
import os

client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY", "sk-WajeURWkrY0fRCdq7fEe65D0146347939f330714A8E0D452"),
    base_url='https://api.open-proxy.cn/v1',
)

In [6]:
MODEL = "gpt-3.5-turbo"

output_file = "./output.json"

with open(output_file, 'w') as fp:
    fp.write("[")
    
    pbar = tqdm(title_column)
    
    flag = 0
    
    for name in title_column:
        pbar.set_description('Processing '+ name)
        
        if flag == 1:
            fp.write(",")
        flag = 1
        
        messages=[]
        messages.append({'role': 'system', 'content': "你是一名出色的汉语言专家，请你对下面给出的词汇提供5个不同的同义词或含义相近的词汇。"})
        messages.append({'role': 'system', 'content': "输入为一个中文词汇。"})
        messages.append({'role': 'system', 'content': "你只能按照所述的格式（JSON对象）进行回复，不要输出多余的换行符和其他内容。它的每个同义词或含义相近的词汇占1行，共5行，注意除最后一行外每行结尾都有一个逗号，每行的格式为：{\"data\":同义词或含义相近的词汇, \"lable\":输入的词汇},"})
        messages.append({'role': 'user', 'content': name})

        response = client.chat.completions.create(
            model = MODEL,
            messages = messages,
            temperature = 0,
        )

        #print(json.dumps(json.loads(response.model_dump_json()), indent=4))

        result = response.choices[0].message.content
        #print(result)
        fp.write("\n")
        fp.write(result)
        
        pbar.update(1)
        
    fp.write("\n]")

  0%|          | 0/2331 [00:00<?, ?it/s]